In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "t-bank-ai/T-lite-instruct-0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
from transformers import AutoModelForCausalLM
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer

dataset = load_dataset("json", data_files="../data/tLiteOneAnswer2Dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
#sft_config = SFTConfig(output_dir="/tmp")

print(dataset.column_names)

trainer = SFTTrainer(
    model,
    train_dataset=dataset
    #args=sft_config,
)

trainer.train()

['id', 'source', 'messages']


/home/user1/environments/hack/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:146: UserWarning: No `SFTConfig` passed, using `output_dir=tmp_trainer`.
  warnings.warn(f"No `SFTConfig` passed, using `output_dir={output_dir}`.")
/home/user1/environments/hack/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/user1/environments/hack/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:276: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/user1/environments/hack/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:353: UserWarning: You passed a `dataset_kwargs` 

Map:   0%|          | 0/476 [00:00<?, ? examples/s]

/home/user1/environments/hack/lib/python3.10/site-packages/accelerate/accelerator.py:457: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 123edfink123 (123edfink123-omstu). Use `wandb login --relogin` to force relogin


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 1 has a total capacty of 31.73 GiB of which 3.44 MiB is free. Including non-PyTorch memory, this process has 31.73 GiB memory in use. Of the allocated memory 31.11 GiB is allocated by PyTorch, and 249.12 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f08341af610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f02b1182c50, execution_count=4 error_before_exec=None error_in_exec=CUDA out of memory. Tried to allocate 16.00 MiB. GPU 1 has a total capacty of 31.73 GiB of which 3.44 MiB is free. Including non-PyTorch memory, this process has 31.73 GiB memory in use. Of the allocated memory 31.11 GiB is allocated by PyTorch, and 249.12 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF info=<ExecutionInfo object at 7f02b1182800, raw_cell="#sft_config = SFTConfig(output_dir="/tmp")

print(.." store_history=True silent=False shell_futures=True cell_id=219983cc-6887-4580-99f7-1ad0bebfc707> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

# Не хватает памяти.

In [ ]:
import subprocess
from pathlib import Path

from datasets import load_dataset

def convert_to_chat_record(row: dict[str, str | int]) -> ChatDatasetRecord:
    for msg in row['messages']:
        if msg['role'] == 'assistant':
            msg['role'] = 'bot'

    return ChatDatasetRecord(
        id=row['id'],
        messages=row['messages'][:-1],
    ).dict()


ds = load_dataset("json", data_files="data/tLiteOneAnswerDataset.json")

ds['train_chat_records'] = ds['train'].map(
    convert_to_chat_record, remove_columns=ds['train'].column_names
)
    # ds['val_chat_records'] = ds['test_prefs'].map(convert_to_chat_record, remove_columns=ds['test_prefs'].column_names)

write_jsonl([item for item in ds['train_chat_records']], Path('train_chat.jsonl'))
    # write_jsonl([item for item in ds['val_chat_records']], Path('val_chat.jsonl'))

launch_code = 'data/sft.json'
subprocess.run(launch_code, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)